In [51]:
import csv
import math
import pandas as pd
import numpy as np
import nvector as nv

import folium
import folium.plugins as plugins
import ipywidgets as widgets
from IPython.display import HTML
from IPython.display import display

reload(folium)
reload(folium.plugins)

#parse and load csv file into a Pandas DataFrame
csvfile = 'Divisions.csv'
df = pd.read_csv(csvfile,parse_dates=['DATE'])

#group the data by Army group and num
grouped_div = df.groupby(['Unit Type','Number'])

#create a list of army groups to use in the selector
army_groups = list(set(map((lambda key: key[0]),grouped_div.groups.keys())))

# remove the Mass Unit type
tmp = set(army_groups)
tmp.remove('Mass')
army_groups = list(tmp)

out = widgets.Output()

def get_div_origin_dest(div_data_df):
    div_data = dict()
    for row in div_data_df.itertuples():
        # only consider non-detachment units
        if math.isnan(row.Weight_D):
            continue
        if row.DATE not in div_data:
            div_data[row.DATE] = dict()
            div_data[row.DATE]['POINT_Y'] = row.POINT_Y
            div_data[row.DATE]['POINT_X'] = row.POINT_X
    div_locs = []
    for key in sorted(div_data):
        div_loc = dict()
        div_loc['POINT_Y'] = div_data[key]['POINT_Y']
        div_loc['POINT_X'] = div_data[key]['POINT_X']
        div_locs.append(div_loc)
    #there is an origin and dest
    if len(div_locs) > 1:
        origin = div_locs[0]
        dest = div_locs[len(div_locs)-1]
        return [[origin['POINT_Y'],origin['POINT_X']],
                [dest['POINT_Y'],dest['POINT_X']]]
    else:
        # no non-detachment movement?
        # look at just the detachments instead
        div_data = dict()
        for row in div_data_df.itertuples():
            # only consider non-detachment units
            if math.isnan(row.Weight_D):
                if row.DATE not in div_data:
                    div_data[row.DATE] = dict()
                    div_data[row.DATE]['POINT_Y'] = row.POINT_Y
                    div_data[row.DATE]['POINT_X'] = row.POINT_X
        div_locs = []
        for key in sorted(div_data):
            div_loc = dict()
            div_loc['POINT_Y'] = div_data[key]['POINT_Y']
            div_loc['POINT_X'] = div_data[key]['POINT_X']
            div_locs.append(div_loc)
        #there is an origin and dest
        if len(div_locs) > 1:
            origin = div_locs[0]
            dest = div_locs[len(div_locs)-1]
            return [[origin['POINT_Y'],origin['POINT_X']],
                    [dest['POINT_Y'],dest['POINT_X']]]
            return None

#gets the vector's bearing from North
def get_bearing(orig, dest):
    wgs84 = nv.FrameE(name='WGS84')
    pointA = wgs84.GeoPoint(latitude=orig[0], longitude=orig[1], degrees=True)
    pointB = wgs84.GeoPoint(latitude=dest[0], longitude=dest[1], degrees=True)
    p_AB_E = nv.delta_E(pointA, pointB)
    frame_N = nv.FrameN(pointA)
    p_AB_N = p_AB_E.change_frame(frame_N)
    p_AB_N = p_AB_N.pvector.ravel()
    azimuth = np.arctan2(p_AB_N[1], p_AB_N[0])
    return np.rad2deg(azimuth)

# event handler for button click
def plot_button_clicked(b):
    if len(group_sel.value) == 0:
        out.clear_output()
        with out:
            display(widgets.HTML(value="<HTML><font size='4' color='red'>Please select atleast one army group</font></HTML>"))
    else:
        plot_average_vectors(group_sel.value)

def plot_average_vectors(sel_groups):
    origin_lats = []
    origin_lons = []
    dest_lats = []
    dest_lons = []

    #iterate over divisions
    for group_key in grouped_div.groups.keys():
        if group_key[0] in sel_groups:
            div_df = grouped_div.get_group(group_key)[['POINT_X','POINT_Y','Weight_D','DATE']]
            #get division's origin and destination
            div_coords = get_div_origin_dest(div_df)

            if div_coords is not None:
                origin_lats.append(div_coords[0][0])
                origin_lons.append(div_coords[0][1])
                dest_lats.append(div_coords[1][0])
                dest_lons.append(div_coords[1][1])

    points = nv.GeoPoint(latitude=origin_lats,longitude=origin_lons,degrees=True)
    nvectors = points.to_nvector()
    n_EM_E = nvectors.mean()
    g_EM_E = n_EM_E.to_geo_point()
    lat, lon = g_EM_E.latitude_deg, g_EM_E.longitude_deg
    mean_origin = [lat[0],lon[0]]

    points = nv.GeoPoint(latitude=dest_lats,longitude=dest_lons,degrees=True)
    nvectors = points.to_nvector()
    n_EM_E = nvectors.mean()
    g_EM_E = n_EM_E.to_geo_point()
    lat, lon = g_EM_E.latitude_deg, g_EM_E.longitude_deg
    mean_dest = [lat[0],lon[0]]

    #now create the map
    div_map = folium.Map([54.78, 32.04],zoom_start=6)

    #first the markers
    folium.CircleMarker(mean_origin,radius=4,color='green').add_to(div_map)
    folium.CircleMarker(mean_dest,radius=4,color='red').add_to(div_map)
    orig_dest_pair = [mean_origin,mean_dest]
    vect_line = folium.PolyLine(locations=orig_dest_pair,weight=1,color='black')
    div_map.add_child(vect_line)

    #get vector direction
    rotation = get_bearing(mean_origin,mean_dest) - 90 #to account for eastward initial rotation
    #now display arrows in the direction
    arrow_lats = np.linspace(mean_origin[0], mean_dest[0], 4)[1:3]
    arrow_lons = np.linspace(mean_origin[1], mean_dest[1], 4)[1:3]
    for points in zip(arrow_lats, arrow_lons):
        folium.RegularPolygonMarker(location=points, 
                                    fill_color='black', number_of_sides=3, 
                                    radius=4, rotation=rotation).add_to(div_map)
    # now display the map
    out.clear_output()
    with out:
        display(div_map)

style = {'description_width': 'initial'}
group_sel = widgets.SelectMultiple(
                options=army_groups,
                rows=8,
                description='Select one or more army group(s):',
                disabled=False,
                style=style
            )
plot_btn = widgets.Button(
                description='Plot Average Vector',
                disabled=False,
                button_style='',
                tooltip='Click to plot'
)
plot_btn.on_click(plot_button_clicked)
sel_ui = widgets.HBox([group_sel,plot_btn])
res = widgets.VBox([sel_ui,out])
res

VkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oU2VsZWN0TXVsdGlwbGUoZGVzY3JpcHRpb249dSdTZWxlY3Qgb25lIG9yIG1vcmUgYXJteSBncm91cChzKTonLCBvcHRpb25zPSgnSW5mIETigKY=
